In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.cluster import AgglomerativeClustering
from scipy.optimize import minimize
import ipywidgets as widgets
from IPython.display import display


class DownloadFailureError(Exception):
    pass

def download_with_early_stop(tickers):
    """
    Télécharge les données avec vérification des échecs après téléchargement
    """
    max_failures = len(tickers) * 0.1  # 10% des tickers
    
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.now() - timedelta(days=366)).strftime('%Y-%m-%d')
    
    try:
        data = yf.download(
            tickers,
            start=start_date,
            end=end_date,
            progress=True,
            timeout=5
        )
        
        # Vérifier les échecs après téléchargement
        failed_tickers = set(tickers) - set(data['Adj Close'].columns)
        if len(failed_tickers) >= max_failures:
            return None, f"Arrêt : {len(failed_tickers)} échecs dépassent le seuil de {max_failures:.0f}. Tickers manquants : {', '.join(failed_tickers)}"
            
        return data, None
    except Exception as e:
        return None, f"Erreur inattendue : {str(e)}"


def get_portfolio_performance(data, tickers, top_n=None, min_hold_days=30, momentum_band=0.05):
    if top_n is None:
        top_n = min(10, len(tickers))

    end_date = datetime.now()
    ytd_start = end_date.replace(month=1, day=1).date()
    
    prices = data.dropna(axis=1, how='all')
    if prices.empty:
        raise ValueError("No valid price data found")
    
    if not isinstance(prices.index, pd.DatetimeIndex):
        prices.index = pd.to_datetime(prices.index)

    end_date = datetime.now()
    ytd_start = end_date.replace(month=1, day=1).date()
    
    if not isinstance(data.index, pd.DatetimeIndex):
        data.index = pd.to_datetime(data.index)
    
    dates = {
        'YTD': min(date for date in data.index.date if date >= ytd_start).strftime('%Y-%m-%d'),
        '1m': data.index[-22].date().strftime('%Y-%m-%d'),
        '1s': data.index[-6].date().strftime('%Y-%m-%d'),
        '1j': data.index[-2].date().strftime('%Y-%m-%d')
    }
    
    days_in_topN = {}
    entry_dates = {}
    consecutive_days = {ticker: 0 for ticker in tickers}
    last_seen = {ticker: False for ticker in tickers}
    
    for day in range(len(data)):
        momentum_day = ((data.iloc[day] / data.iloc[0] - 1) * 100).sort_values(ascending=False)
        top_day = set(momentum_day.head(top_n).index)
        
        for ticker in tickers:
            if ticker in top_day:
                if not last_seen[ticker]:
                    entry_dates[ticker] = data.index[day]
                consecutive_days[ticker] += 1
                last_seen[ticker] = True
            else:
                if last_seen[ticker]:
                    days_in_topN[ticker] = consecutive_days[ticker]
                consecutive_days[ticker] = 0
                last_seen[ticker] = False
    
    for ticker in tickers:
        if last_seen[ticker]:
            days_in_topN[ticker] = consecutive_days[ticker]
    
    momentum_365 = ((data.iloc[-1] / data.iloc[0] - 1) * 100).sort_values(ascending=False)
    momentum_prev = ((data.iloc[-2] / data.iloc[0] - 1) * 100).sort_values(ascending=False)
    
    # Vérifier la période de détention minimale
    current_holdings = set()
    for ticker in momentum_prev.head(top_n).index:
        if ticker in days_in_topN and days_in_topN[ticker] < min_hold_days:
            current_holdings.add(ticker)
    
    # Appliquer la bande de roulement
    potential_exits = set()
    for ticker in current_holdings:
        current_momentum = momentum_365[ticker]
        threshold = momentum_365.iloc[top_n-1] * (1 - momentum_band)
        if current_momentum < threshold:
            potential_exits.add(ticker)
    
    # Ne sortir que les positions ayant dépassé la période minimale
    final_exits = set()
    for ticker in potential_exits:
        if days_in_topN.get(ticker, 0) >= min_hold_days:
            final_exits.add(ticker)
    
    # Mettre à jour le top_n actuel
    current_top = momentum_365.head(top_n)
    maintained_positions = current_holdings - final_exits
    new_positions = set(current_top.index) - maintained_positions
    
    # Limiter les nouvelles positions pour respecter le top_n
    available_slots = top_n - len(maintained_positions)
    if available_slots > 0:
        new_positions = set(list(new_positions)[:available_slots])
    
    final_positions = maintained_positions.union(new_positions)
    final_top = momentum_365[list(final_positions)]
    
    perf_since_entry = {}
    for ticker in final_top.index:
        if ticker in entry_dates:
            entry_price = data[ticker].loc[entry_dates[ticker]]
            current_price = data[ticker].iloc[-1]
            perf_since_entry[ticker] = ((current_price / entry_price - 1) * 100)
    
    portfolio_data = data[final_top.index]
    
    perf = {
        '1a': (portfolio_data.iloc[-1] / portfolio_data.iloc[0] - 1).mean() * 100
    }
    
    for period, ref_date_str in dates.items():
        ref_date = pd.to_datetime(ref_date_str).date()
        ref_idx = portfolio_data.index[portfolio_data.index.date <= ref_date][-1]
        ref_data = portfolio_data.loc[ref_idx]
        perf[period] = (portfolio_data.iloc[-1] / ref_data - 1).mean() * 100
    
    val_100 = 100
    valorisations = {k: val_100 / (1 + v/100) for k, v in perf.items()}
    valorisations['Actuelle'] = val_100
    
    return final_top, perf, valorisations, new_positions, final_exits, days_in_topN, data, perf_since_entry

def optimize_vol_constrained(allocation, returns, max_vol):
    def portfolio_vol(weights):
        return np.sqrt(252) * (returns * weights).sum(axis=1).std()
    
    def objective(weights):
        return -((returns * weights).sum(axis=1).mean() * 252)
    
    constraints = [
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'ineq', 'fun': lambda x: max_vol - portfolio_vol(x)}
    ]
    bounds = [(0, 0.2) for _ in range(len(returns.columns))]
    
    result = minimize(objective, allocation/sum(allocation), 
                    constraints=constraints, bounds=bounds, method='SLSQP')
    return result.x * sum(allocation)

def get_balanced_portfolio(data, tickers, total_amount=1000, n_clusters=10, 
                         momentum_weight=0.6, sharpe_weight=0.4, max_vol=0.25,
                         min_position_weight=0.02):
    # Vérifier la présence de données valides
    if data.empty:
        raise ValueError("No data available")
        
    # Obtenir uniquement les tickers valides avec des données non-NaN
    adj_close = data['Adj Close'].dropna(axis=1, how='all')
    volume = data['Volume'].dropna(axis=1, how='all')
    
    if adj_close.empty or volume.empty:
        raise ValueError("No valid price data found")
        
    valid_tickers = adj_close.columns.intersection(volume.columns)
    if len(valid_tickers) == 0:
        raise ValueError("No valid tickers found")
    
    # Calculer les rendements
    returns = adj_close[valid_tickers].pct_change(fill_method=None).fillna(0)
    mean_volume = volume[valid_tickers].mean()
    
    # Filtrer les actions liquides
    liquid_stocks = mean_volume[mean_volume > mean_volume.quantile(0.2)].index
    if len(liquid_stocks) == 0:
        raise ValueError("No liquid stocks found")
        
    liquid_returns = returns[liquid_stocks]
    
    distances = 1 - liquid_returns.corr()
    clusters = pd.Series(AgglomerativeClustering(n_clusters=min(n_clusters, len(liquid_stocks)), 
                                               distance_threshold=None,
                                               metric='precomputed',
                                               linkage='single')
                       .fit_predict(distances), index=liquid_returns.columns)
    
    vol = liquid_returns.std() * np.sqrt(252)
    momentum = ((data['Adj Close'][liquid_stocks].iloc[-1] / 
                data['Adj Close'][liquid_stocks].iloc[0] - 1) * 100).fillna(0)
    sharpe = (liquid_returns.mean() * 252) / vol
    
    cluster_weights = {}
    n_clusters = clusters.max() + 1
    for i in range(n_clusters):
        cluster_assets = clusters[clusters == i].index
        if len(cluster_assets) > 0:
            cluster_momentum = momentum[cluster_assets].clip(-100, 100).mean()
            cluster_sharpe = sharpe[cluster_assets].clip(-3, 3).mean()
            cluster_weights[i] = max(0, np.nansum([momentum_weight * cluster_momentum, 
                                                sharpe_weight * cluster_sharpe]))
    
    allocation = pd.Series(0.0, index=liquid_stocks)
    if sum(cluster_weights.values()) > 0:
        for i, weight in cluster_weights.items():
            cluster_assets = clusters[clusters == i].index
            if len(cluster_assets) > 0:
                asset_scores = (0.7 * momentum[cluster_assets].rank() + 
                              0.3 * sharpe[cluster_assets].rank()).clip(0, None)
                if asset_scores.sum() > 0:
                    allocation[cluster_assets] = total_amount * float(weight/sum(cluster_weights.values())) * (asset_scores / asset_scores.sum())
    
    # Appliquer le poids minimum par position
    allocation[allocation < min_position_weight * total_amount] = 0
    if allocation.sum() > 0:
        allocation = allocation / allocation.sum() * total_amount
    
    nonzero_allocation = allocation[allocation > 0]
    if len(nonzero_allocation) > 0:
        nonzero_returns = liquid_returns[nonzero_allocation.index]
        optimized = optimize_vol_constrained(nonzero_allocation, nonzero_returns, max_vol)
        allocation = pd.Series(optimized, index=nonzero_returns.columns)
    
    final_allocation = pd.Series(0.0, index=tickers)
    final_allocation[allocation.index] = allocation
    
    port_weights = final_allocation / total_amount
    metrics = {
        'return': (returns * port_weights).sum(axis=1).mean() * 252,
        'volatility': (returns * port_weights).sum(axis=1).std() * np.sqrt(252),
        'sharpe': ((returns * port_weights).sum(axis=1).mean() * 252) / 
                 ((returns * port_weights).sum(axis=1).std() * np.sqrt(252))
    }
    
    return final_allocation.round(2), metrics, clusters

def interactive_portfolio_analysis(TICKERS, min_hold_days=30, momentum_band=0.05, min_position_weight=0.02):
    top_n_slider = widgets.IntSlider(
        value=10,
        min=1,
        max=len(TICKERS),
        step=1,
        description='Top N Stocks:',
        continuous_update=False
    )
    
    clusters_slider = widgets.IntSlider(
        value=10,
        min=1,
        max=len(TICKERS),
        step=1,
        description='Clusters:',
        continuous_update=False
    )
    
    amount_text = widgets.FloatText(
        value=1000,
        description='Total Amount (€):',
        style={'description_width': 'initial'}
    )

    momentum_weight_text = widgets.FloatText(
        value=0.6,
        description='Momentum Weight:',
        style={'description_width': 'initial'}
    )

    sharpe_weight_text = widgets.FloatText(
        value=0.4,
        description='Sharpe Weight:',
        style={'description_width': 'initial'}
    )

    max_vol_text = widgets.FloatText(
        value=0.25,
        description='Max Volatility:',
        style={'description_width': 'initial'}
    )
    
    analyze_button = widgets.Button(description="Analyze Portfolio")
    output = widgets.Output()
    
    def on_analyze_button_clicked(b):
        with output:
            output.clear_output()

            print("📥 Téléchargement des données...")
            data, error = download_with_early_stop(TICKERS)

            if error:
                print("\n⚠️ ERREUR DE TÉLÉCHARGEMENT")
                print("=" * 80)
                print(error)
                return

            top_n = top_n_slider.value
            n_clusters = clusters_slider.value
            total_amount = amount_text.value
            momentum_weight = momentum_weight_text.value
            sharpe_weight = sharpe_weight_text.value
            max_vol = max_vol_text.value

            print(f"\n📊 ANALYSE MOMENTUM TOP {top_n}")
            print("=" * 80)

            print(f"\n📊 ANALYSE MOMENTUM TOP {top_n}")
            print("=" * 80)
            
            print(f"\n📊 ANALYSE MOMENTUM TOP {top_n}")
            print("=" * 80)
            
            top_stocks, perf, vals, entries, exits, days_in_top, momentum_data, perf_since_entry = get_portfolio_performance(
                data['Adj Close'], TICKERS, top_n, min_hold_days, momentum_band
            )
            
            print("\n🏆 Classement Momentum 365j:")
            print(f"{'#':2} {'Ticker':6} {'Momentum':>9} {'Jours Top':>11} {'Cours':>10} {'Perf.Entry':>10}")
            print("-" * 80)
            for i, (ticker, momentum) in enumerate(top_stocks.items(), 1):
                current_price = momentum_data[ticker].iloc[-1]
                perf_entry = perf_since_entry.get(ticker, 0)
                print(f"{i:2d} {ticker:6} {momentum:8.1f}% {days_in_top.get(ticker,0):8d}j {current_price:10.2f}€ {perf_entry:9.1f}%")
            
            print("\n📈 Performance du portefeuille:")
            print("-" * 40)
            print(f"{'Actuel':12} : {'---':>6} (val. {vals['Actuelle']:6.2f})")
            
            periods = {'1j': '1 jour', '1s': '1 semaine', '1m': '1 mois', '1a': '1 an', 'YTD': 'Depuis 01/01'}
            for p in ['1j', '1s', '1m', 'YTD', '1a']:
                print(f"{periods[p]:12} : {perf[p]:6.2f}% (val. {vals[p]:6.2f})")
            
            if entries or exits:
                print("\n🔄 Changements depuis hier:")
                if entries:
                    print(f"  Entrées : {', '.join(entries)}")
                if exits:
                    print(f"  Sorties : {', '.join(exits)}")
            
            print("\n\n💼 ANALYSE PORTEFEUILLE ÉQUILIBRÉ")
            print("=" * 80)
            
            allocation, metrics, clusters = get_balanced_portfolio(
                data, TICKERS, total_amount, n_clusters,
                momentum_weight=momentum_weight,
                sharpe_weight=sharpe_weight,
                max_vol=max_vol,
                min_position_weight=min_position_weight
            )
            
            print("\n🔍 Clusters identifiés:")
            for i in range(clusters.max() + 1):
                print(f"\nCluster {i}: {', '.join(clusters[clusters == i].index)}")

            print("\n💰 Allocation optimisée (€):")
            print("-" * 40)
            for ticker, amount in allocation.sort_values(ascending=False).items():
                if amount > 0:
                    print(f"{ticker:6}: {amount:7.1f}€")

            print("\n📊 Métriques du portefeuille:")
            print("-" * 40)
            print(f"Sharpe     : {metrics['sharpe']:.2f}")
            print(f"Volatilité : {metrics['volatility']*100:.1f}%")
            print(f"Rendement  : {metrics['return']*100:.1f}%")
    
    analyze_button.on_click(on_analyze_button_clicked)
    
    # Display widgets
    display(top_n_slider, clusters_slider, amount_text, momentum_weight_text, sharpe_weight_text, max_vol_text, analyze_button, output)


# Usage example (commented out)
TICKERS = [
    "MC.PA","OR.PA","SU.PA","AIR.PA","TTE.PA","SAN.PA","CDI.PA","EL.PA","SAF.PA","AI.PA","BNP.PA","CS.PA","AXA SA","DG.PA","DSY.PA","SGO.PA","BN.PA","ACA.PA","ENGI.PA","KER.PA","HO.PA","CAP.PA","RI.PA","LR.PA","ORA.PA","PUB.PA",
    "GLE.PA","ML.PA","DIM.PA","VIE.PA","AM.PA","BOL.PA","RNO.PA","BVI.PA","AMUN.PA","BIM.PA","AC.PA","EN.PA","ENX.PA","ADP.PA","URW.PA","SW.PA","IPN.PA","RNL.PA","ERF.PA","ALO.PA","CA.PA","FGR.PA","LI.PA","GET.PA","EDEN.PA","RXL.PA","IAM.PA",
    "CBDG.PA","GFC.PA","FDJ.PA","ODET.PA","COTY.PA","AKE.PA","AYV.PA","RF.PA","COV.PA","Covivio","GTT.PA","SPIE.PA","SPIE SA","TEP.PA","SK.PA","SEB SA","TE.PA","ELIS.PA","Elis SA","SCR.PA","SCOR SE","VK.PA","NEX.PA","MF.PA","MLHK.PA","H&K AG",
    "TKO.PA","FLY.PA","SOP.PA","DEC.PA","PLX.PA","ITP.PA","VRLA.PA","SOI.PA","RCO.PA","COVH.PA","MMB.PA","ATE.PA","VU.PA","FR.PA","IDL.PA","BB.PA","RUI.PA","VIRP.PA","TRI.PA","BAIN.PA","VIV.PA","COFA.PA","CARM.PA","LOUP.PA","NK.PA","WLN.PA",
    "ALTA.PA","UNBL.PA","IPS.PA","CAF.PA","AF.PA","PLNW.PA","CBE.PA","VCT.PA","PEUG.PA","RBT.PA","EXN.PA","STF.PA","STEF SA","ICAD.PA","SESG.PA","OPM.PA","ERA.PA","ARG.PA","TFI.PA","TF1 SA","UBI.PA","OVH.PA","MMT.PA","ES.PA","BLV.PA","MAU.PA",
    "GDS.PA","FII.PA","WAVE.PA","EXOSENS","CRLA.PA","NRO.PA","LSS.PA","MERY.PA","EMEIS","ETL.PA","ELEC.PA","FREY.PA","Frey SA","DBG.PA","CNDF.PA","LTA.PA","CDA.PA","FNAC.PA","MTU.PA","VETO.PA","TKTT.PA","VIL.PA","BEN.PA","EC.PA","VAC.PA","SAVE.PA",
    "BASS.PA","NXI.PA","XFAB.PA","SDG.PA","THEP.PA","CRAV.PA","CRSU.PA","CRAP.PA","CEN.PA","SCHP.PA","TFF.PA","KOF.PA","QDT.PA","LPE.PA","SBT.PA","EQS.PA","BUR.PA","AUB.PA","GLO.PA","NVDA","AAPL","MSFT","AMZN","GOOGL","GOOG","META","TSLA","AVGO",
    "BRK-B","ORCL","TCEHY","TCTZF","NFLX","COST","NONOF","LVMHF","LVMUY","JPM-PD","JPM-PC","BML-PG","SAPGF","BML-PH","BML-PL","BAC-PE","IDCBY","BAC-PK","SSNLF","ABBV","IDCBF","HESAY","ASMLF","ASML","GDVTZ","ACGBF","TMUS","BML-PJ","BAC-PB","RHHBF",
    "CSCO","RHHVF","RHHBY","TOYOF","ACGBY","BABAF","AZNCF","BABA","NSRGY","NSRGF","ISRG","CICHY","RYDAF","BACHY","LRLCY","WFC-PY","NVSEF","BACHF","SHEL","CICHF","LRLCF","PCCYF","ADBE","QCOM","HBCYF","HSBC","CMWAY","PLTR","SIEGY","SMAWF","INTU","ANET",
    "CBAUF","IDEXF","SBGSF","SPGI","IDEXY","SBGSY","MBFJF","DTEGF","AMAT","DTEGY","FMXUF","SCHW","CIHKY","MUFG","AMGN","UBER","CMCSA","UNLYF","SHOP","EADSF","EADSY","BHPLF","SNYNF","TTFNF","SNEJF","CHDRY","CILJF","SONY","CHDRF","WFC-PC","ALIZF","ALIZY",
    "HTHIF","ESLOY","RTNTF","HTHIY","MPNGF","Meituan","ESLOF","CIHHF","XIACY","MPNGY","PNGAY","XIACF","CIIHF","GILD","PIAIF","VRTX","BYDDF","BYDDY","SBUX","SAFRF","SAFRY","CFRUY","CUAEF","CFRHF","ABBNY","ABLZF","MRVL","KYCCF","RCRUY","UNCFF","UNCRY",
    "CSUAY","SPOT","FRCOF","RCRRF","LRCX","KLAC","SMFNF","SNPMF","SFTBF","FRCOY","AIQUY","AIQUF","SFTBY","SMFG","RTPPF","BUDFF","USB-PH","IBKR","CRWD","DBSDY","RLXXF","EQIX","RELX","INTC","INFY","IVSXF","PYPL","EBBNF","DBSDF","PROSF","PROSY","GS-PA",
    "IBDRY","IBDSF","CDNS","NPPXF","IVSBF","ZFSVF","MS-PA","ZURVY","WELL","BNPQF","BNPQY","SNPS","ATLCY","TOELF","BPAQF","AXAHF","AXAHY","CSLLY","MSTR","CMXHF","ATLKY","MS-PK","TOELY","GS-PD","MS-PI","PBR-A","NTTYY","MS-PF","IITSF","BTAFF","BCDRF",
    "DELL","CTAS","ABNB","MS-PE","AAIGF","LNSTY","LDNXF","ISNPY","RACE","MDLZ","HNHPF","AAGIY","SCCO","DASH","CGXYY","PBCRF","USB-PP","COIN","NTDOF","SBKFF","FTNT","REGN","NTDOY","MURGY","MURGF","PBCRY","ESOCF","WEBNF","ENLAY","NABZY","BKFCF","PSTVY",
    "GLAXF","BCMXY","TEAM","CHGCY","TKOMF","CHGCF","PSBKF","WDAY","ITOCF","RLLCF","MKGAF","NTES","MKKGY","SHECY","BBVXF","ITOCY","STOHF","RBSPF","TKOMY","DGEAF","EQNR","PPWLM","ADSK","BBVA","RYCEF","DBK.DE","EI.PA","ENEL.MI","FRE.DE","IBE.MC","INGA.AS",
    "ISP.MI","EOAN.DE","G.MI","ALV.DE","BBVA.MC","BAYN.DE","ABI.BR","ENI.MI","BMW.DE","ASML.AS","DTE.DE","BAS.DE","EURUSD=X","MT.AS","RMS.PA","STLAP.PA","STMPA.PA","V","JPM","JNJ","WMT","PG","MA","DIS","HD","VZ","UNH","KO","PFE","XOM","MRK","NKE","ABT",
    "PEP","CRM","TMO","MDT","LLY","MS","BA","STLA","NESN.SW","A","AAL","AAP","ACN","ADI","ADM","ADP","AEE","AEP","AES","AFL","AIG","AIV","AIZ","AJG","AKAM","ALB","ALGN","ALK","ALL","ALLE","AMD","AME","AMP","AMT","ANSS","AON","AOS","APA","APD","APH","APTV",
    "ARE","ATO","AVB","AVY","AWK","AXP","AZO","BAC","BAX","BBY","BDX","BEN","BIIB","BK","BKNG","BKR","BLK","BMY","BR","BSX","BWA","BXP","C","CAG","CAH","CAT","CB","CBOE","CBRE","CCI","CCL","CE","CF","CFG","CHD","CHRW","CHTR","CI","CINF","CL","CLX","CMA","CME",
    "CMG","CMI","CMS","CNC","CNP","COF","COO","COP","CPB","CPRT","CSX","CVS","CVX","D","DAL","DD","DE","DFS","DG","DGX","DHI","DHR","DLR","DLTR","DOV","DOW","DRI","DTE","DUK","DVA","DVN","DXC","EA","EBAY","ECL","ED","EFX","EIX","EL","EMN","EMR","EOG","EQR","ES",
    "ESS","ETN","ETR","EVRG","EW","EXC","EXPD","EXPE","EXR","F","FANG","FAST","FCX","FDX","FE","FFIV","FIS","FITB","FLR","FLS","FMC","FOX","FOXA","FTV","GD","GE","GIS","GL","GLW","GM","GPC","GPN","GS","GWW","HAL","HAS","HBAN","HBI","HCA","HCP","HES","HIG","HII",
    "HLT","HOG","HOLX","HON","HP","HPE","HPQ","HRB","HRL","HSIC","HST","HSY","HUM","IBM","ICE","IDXX","IEX","IFF","ILMN","INCY","INFO","IP","IPG","IPGP","IQV","IR","IRM","IT","ITW","IVZ","J","JBHT","JCI","JKHY","JNPR","JWN","K","KEY","KEYS","KHC","KIM","KMB",
    "KMI","KMX","KR","KSS","L","LB","LDOS","LEG","LEN","LH","LHX","LIN","LKQ","LMT","LNC","LNT","LOW","LUV","LW","LYB","M","MAA","MAC","MAR","MAS","MCD","MCHP","MCK","MCO","MET","MGM","MHK","MKC","MKTX","MLM","MMC","MMM","MNST","MO","MOS","MPC","MSCI","MSI",
    "MTB","MTD","MU","NAVI","NCLH","NDAQ","NEE","NEM","NI","NOC","NOV","NRG","NSC","NTAP","NTRS","NUE","NVR","NWL","NWS","NWSA","O","ODFL","OKE","OMC","ORLY","OXY","PAYC","PAYX","PCAR","PEG","PFG","PGR","PH","PHM","PLD","PM","PNC","PNR","PNW","PPG","PPL",
    "PRGO","PRU","PSA","PSX","PVH","PWR","QRVO","RCL","REG","RF","RHI","RJF","RL","RMD","ROK","ROL","ROP","ROST","RSG","RTX","SBAC","SEE","SHW","SJM","SLB","SLG","SNA","SO","SPG","SRE","STE","STT","STX","STZ","SWK","SWKS","SYY","T","TAP","TDG","TEL","TER",
    "TFC","TFX","TGT","TJX","TPR","TRMB","TROW","TRV","TSCO","TSN","TT","TTWO","TXN","TXT","TYL","UA","UAA","UAL","UDR","UHS","ULTA","UNM","UNP","UPS","URI","USB","VFC","VLO","VMC","VNO","VRSK","VRSN","VTR","WAB","WAT","WBA","WDC","WEC","WFC","WHR","WM",
    "WMB","WRB","WU","WY","WYNN","XEL","XRAY","XRX","XYL","YUM","ZBH","ZBRA","ZION","ZTS","ADS.DE","CON.DE","DB1.DE","LHA.DE","LIN.DE","MUV2.DE","RWE.DE","SAP.DE","SIE.DE","VOW3.DE","ZAL.DE","AAL.L","ABF.L","ADM.L","AHT.L","AV.L","BA.L","BARC.L","BATS.L",
    "BP.L","BTI","CNA.L","DGE.L","GSK.L","HSBA.L","IMB.L","ITV.L","LGEN.L","LLOY.L","RDSA.VI","PUM.DE","NOKIA.HE","2388.HK","1398.HK","600519.SS","601988.SS","601288.SS","601318.SS","000651.SZ","002475.SZ","BHP.AX","CBA.AX","TLS.AX",
    "WBC.AX","CSL.AX","NAB.AX","ANZ.AX","RIO.AX","QBE.AX","WOW.AX","S32.AX","FMG.AX","MQG.AX","TD.TO","RY.TO","BNS.TO","ENB.TO","SU.TO","CNQ.TO","BMO.TO","SHOP.TO","SLF.TO","MFC.TO","PPL.TO","TRP.TO","TSM","SAP","VOW.DE",
]
interactive_portfolio_analysis(TICKERS, min_hold_days=10, momentum_band=0.05, min_position_weight=0.02)

IntSlider(value=10, continuous_update=False, description='Top N Stocks:', max=931, min=1)

IntSlider(value=10, continuous_update=False, description='Clusters:', max=931, min=1)

FloatText(value=1000.0, description='Total Amount (€):', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.6, description='Momentum Weight:', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.4, description='Sharpe Weight:', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.25, description='Max Volatility:', style=DescriptionStyle(description_width='initial'))

Button(description='Analyze Portfolio', style=ButtonStyle())

Output()

In [ ]:
pip install --upgrade ipywidgets